In [ ]:
# 導入必要的模組
import cv2
import mediapipe as mp
import math
import numpy as np
from enum import Enum
from typing import List, Tuple, Optional


In [ ]:
# 定義枚舉類型
class FingerState(Enum):
    """手指狀態枚舉"""
    EXTENDED = 0  # 伸直
    BENT = 1      # 彎曲

class HandGesture(Enum):
    """手勢類型枚舉"""
    GOOD = "good"
    NO = "no!!!"
    ROCK = "ROCK!"
    ZERO = "0"
    ONE = "1"
    TWO = "2"
    THREE = "3"
    FOUR = "4"
    FIVE = "5"
    SIX = "6"
    SEVEN = "7"
    EIGHT = "8"
    NINE = "9"
    OK = "ok"
    PINK = "pink"
    UNKNOWN = ""


In [ ]:
# 設定參數
class Config:
    """設定常數"""
    # 影像設定
    FRAME_WIDTH = 540
    FRAME_HEIGHT = 310
    
    # 手指角度閾值
    FINGER_ANGLE_THRESHOLD = 50
    
    # MediaPipe 設定
    MODEL_COMPLEXITY = 0
    MIN_DETECTION_CONFIDENCE = 0.5
    MIN_TRACKING_CONFIDENCE = 0.5
    
    # 顯示設定
    FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
    LINE_TYPE = cv2.LINE_AA
    TEXT_COLOR = (255, 255, 255)
    TEXT_THICKNESS = 10
    TEXT_SCALE = 5
    TEXT_POSITION = (30, 120)
    
    # 馬賽克設定
    MOSAIC_SIZE = 8
    MOSAIC_PADDING = 10


In [ ]:
# 角度計算器
class AngleCalculator:
    """角度計算器"""
    
    @staticmethod
    def vector_2d_angle(v1: Tuple[float, float], v2: Tuple[float, float]) -> float:
        """計算兩個二維向量之間的角度"""
        v1_x, v1_y = v1
        v2_x, v2_y = v2
        
        try:
            dot_product = v1_x * v2_x + v1_y * v2_y
            magnitude_v1 = math.sqrt(v1_x**2 + v1_y**2)
            magnitude_v2 = math.sqrt(v2_x**2 + v2_y**2)
            cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
            angle = math.degrees(math.acos(cos_angle))
        except (ZeroDivisionError, ValueError):
            angle = 180
        
        return angle
    
    @staticmethod
    def calculate_finger_angles(landmarks: List[Tuple[float, float]]) -> List[float]:
        """計算五個手指的角度"""
        angles = []
        
        # 定義手指節點索引
        finger_indices = [
            (0, 2, 3, 4),   # 大拇指
            (0, 6, 7, 8),   # 食指
            (0, 10, 11, 12), # 中指
            (0, 14, 15, 16), # 無名指
            (0, 18, 19, 20)  # 小拇指
        ]
        
        for base_idx, joint1_idx, joint2_idx, tip_idx in finger_indices:
            v1 = (
                int(landmarks[base_idx][0]) - int(landmarks[joint1_idx][0]),
                int(landmarks[base_idx][1]) - int(landmarks[joint1_idx][1])
            )
            v2 = (
                int(landmarks[joint2_idx][0]) - int(landmarks[tip_idx][0]),
                int(landmarks[joint2_idx][1]) - int(landmarks[tip_idx][1])
            )
            
            angle = AngleCalculator.vector_2d_angle(v1, v2)
            angles.append(angle)
        
        return angles


In [ ]:
# 手勢識別器
class GestureRecognizer:
    """手勢識別器"""
    
    def __init__(self):
        self.gesture_patterns = {
            # (大拇指, 食指, 中指, 無名指, 小拇指) - True表示伸直, False表示彎曲
            HandGesture.GOOD: (True, False, False, False, False),
            HandGesture.NO: (False, False, True, False, False),
            HandGesture.ROCK: (True, True, False, False, True),
            HandGesture.ZERO: (False, False, False, False, False),
            HandGesture.ONE: (False, True, False, False, False),
            HandGesture.TWO: (False, True, True, False, False),
            HandGesture.THREE: (False, True, True, True, False),
            HandGesture.FOUR: (False, True, True, True, True),
            HandGesture.FIVE: (True, True, True, True, True),
            HandGesture.SIX: (True, False, False, False, True),
            HandGesture.SEVEN: (True, True, False, False, False),
            HandGesture.EIGHT: (True, True, True, False, False),
            HandGesture.NINE: (True, True, True, True, False),
            HandGesture.OK: (False, False, True, True, True),
            HandGesture.PINK: (False, False, False, False, True),
        }
    
    def _angles_to_finger_states(self, angles: List[float]) -> Tuple[bool, ...]:
        """將角度轉換為手指狀態"""
        return tuple(angle < Config.FINGER_ANGLE_THRESHOLD for angle in angles)
    
    def recognize(self, finger_angles: List[float]) -> HandGesture:
        """識別手勢"""
        finger_states = self._angles_to_finger_states(finger_angles)
        
        for gesture, pattern in self.gesture_patterns.items():
            if finger_states == pattern:
                return gesture
        
        return HandGesture.UNKNOWN


In [ ]:
# 效果處理器
class EffectProcessor:
    """效果處理器"""
    
    @staticmethod
    def apply_mosaic(image: np.ndarray, x_min: int, y_min: int, 
                    x_max: int, y_max: int) -> np.ndarray:
        """應用馬賽克效果"""
        if x_min >= x_max or y_min >= y_max:
            return image
        
        # 提取區域
        region = image[y_min:y_max, x_min:x_max]
        if region.size == 0:
            return image
        
        # 應用馬賽克
        height, width = region.shape[:2]
        mosaic = cv2.resize(region, (Config.MOSAIC_SIZE, Config.MOSAIC_SIZE), 
                          interpolation=cv2.INTER_LINEAR)
        mosaic = cv2.resize(mosaic, (width, height), 
                          interpolation=cv2.INTER_NEAREST)
        
        # 將馬賽克區域貼回原圖
        image[y_min:y_max, x_min:x_max] = mosaic
        return image
    
    @staticmethod
    def draw_text(image: np.ndarray, text: str) -> np.ndarray:
        """在圖像上繪製文字"""
        cv2.putText(image, text, Config.TEXT_POSITION, Config.FONT_FACE,
                   Config.TEXT_SCALE, Config.TEXT_COLOR, Config.TEXT_THICKNESS,
                   Config.LINE_TYPE)
        return image


In [ ]:
# 手部追蹤器
class HandTracker:
    """手部追蹤器"""
    
    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            model_complexity=Config.MODEL_COMPLEXITY,
            min_detection_confidence=Config.MIN_DETECTION_CONFIDENCE,
            min_tracking_confidence=Config.MIN_TRACKING_CONFIDENCE
        )
        self.angle_calculator = AngleCalculator()
        self.gesture_recognizer = GestureRecognizer()
        self.effect_processor = EffectProcessor()
    
    def extract_landmarks(self, hand_landmarks) -> Tuple[List[Tuple[float, float]], List[int], List[int]]:
        """提取手部特徵點座標"""
        landmarks = []
        x_coords = []
        y_coords = []
        
        for landmark in hand_landmarks.landmark:
            x = landmark.x * Config.FRAME_WIDTH
            y = landmark.y * Config.FRAME_HEIGHT
            landmarks.append((x, y))
            x_coords.append(int(x))
            y_coords.append(int(y))
        
        return landmarks, x_coords, y_coords
    
    def get_bounding_box(self, x_coords: List[int], y_coords: List[int]) -> Tuple[int, int, int, int]:
        """獲取手部邊界框"""
        x_min = max(0, min(x_coords) - Config.MOSAIC_PADDING)
        y_min = max(0, min(y_coords) - Config.MOSAIC_PADDING)
        x_max = min(Config.FRAME_WIDTH, max(x_coords))
        y_max = min(Config.FRAME_HEIGHT, max(y_coords))
        
        return x_min, y_min, x_max, y_max
    
    def process_frame(self, frame: np.ndarray) -> np.ndarray:
        """處理單一幀圖像"""
        # 轉換顏色空間
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # 提取特徵點
                landmarks, x_coords, y_coords = self.extract_landmarks(hand_landmarks)
                
                if landmarks:
                    # 計算手指角度
                    finger_angles = self.angle_calculator.calculate_finger_angles(landmarks)
                    
                    # 識別手勢
                    gesture = self.gesture_recognizer.recognize(finger_angles)
                    
                    # 根據手勢應用效果
                    if gesture == HandGesture.NO:
                        x_min, y_min, x_max, y_max = self.get_bounding_box(x_coords, y_coords)
                        frame = self.effect_processor.apply_mosaic(frame, x_min, y_min, x_max, y_max)
                    elif gesture != HandGesture.UNKNOWN:
                        frame = self.effect_processor.draw_text(frame, gesture.value)
        
        return frame
    
    def __del__(self):
        """清理資源"""
        if hasattr(self, 'hands'):
            self.hands.close()


In [4]:
# 主應用程式類別
class HandGestureApp:
    """手勢識別應用主類"""
    
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        self.hand_tracker = HandTracker()
        
        if not self.cap.isOpened():
            raise RuntimeError("無法開啟攝影機")
    
    def run(self):
        """運行應用程式"""
        print("手勢識別應用已啟動，按 'q' 鍵退出")
        
        try:
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    print("無法讀取攝影機畫面")
                    break
                
                # 調整畫面大小
                frame = cv2.resize(frame, (Config.FRAME_WIDTH, Config.FRAME_HEIGHT))
                
                # 處理手勢識別
                processed_frame = self.hand_tracker.process_frame(frame)
                
                # 顯示結果
                cv2.imshow('Hand Gesture Recognition', processed_frame)
                
                # 檢查退出條件
                if cv2.waitKey(5) & 0xFF == ord('q'):
                    break
                    
        except KeyboardInterrupt:
            print("程式被中斷")
        finally:
            self.cleanup()
    
    def cleanup(self):
        """清理資源"""
        self.cap.release()
        cv2.destroyAllWindows()


In [5]:
# 運行應用程式
if __name__ == "__main__":
    app = HandGestureApp()
    app.run()


手勢識別應用已啟動，按 'q' 鍵退出
